In [1]:
import pandas as pd
import numpy as np
from pandas.parser import CParserError
from __future__ import division 

In [2]:
data = pd.read_csv('./Dataset/final data.csv', index_col = 0)

In [3]:
change_column = ['default_profile', 'default_profile_image', 'geo_enabled', 'profile_use_background_image', 'profile_background_tile',
                 'is_translator', 'follow_request_sent', 'protected', 'verified', 'notifications', 'contributors_enabled', 'following']

for i in change_column:
    change = np.where(data[i] ==  True, 1, 0)
    data[i] = change

### Extract features

In [10]:
sample = pd.read_csv('sample_data.csv', index_col = 0)
ids = sample['id']

In [11]:
feature6 = pd.read_csv('feature_6.csv', index_col = 0)

In [12]:
following_rate = feature6['following'] / feature6['age']

In [13]:
feature6['following_rate'] = following_rate

In [14]:
# feature 6 for sample
sample_f6 = feature6[feature6.id.isin(ids)]

In [15]:
import os

In [16]:
os.chdir('/Users/Aaron/Desktop/STA160')

In [17]:
sample_names = [i + '_tweets.csv' for i in list(ids.astype(str))]

In [160]:
# get the samples
path =r'/Users/Aaron/Desktop/Sample/'

sample_tw = []
for i in sample_names:
    try:
        address = path + i
        df = pd.read_csv(address, encoding = 'utf-8', index_col = 0)
        sample_tw.append(df)
    except (CParserError, IOError): 
        sample_tw.append([])

In [161]:
def tweet_rate(data):
    #dates = [i.rsplit()[0] for i in data['created_at']]
    dates = []
    
    try:
        for i in data['Created_At']:
            try:
                dates.append(i.rsplit()[0])
            except AttributeError:
                pass

        try:
            total_dates = len(set(dates))
            tweet_rate = len(dates)/total_dates
        
            return tweet_rate
        except ZeroDivisionError:
            return 'None'
    
    except TypeError:
        
        return 'None'


In [162]:
# get the sample tweet rates
sample_tr = [tweet_rate(sample_tw[i]) for i in range(len(sample_tw))]    
sample_tr = pd.Series(sample_tr)

In [163]:
# reset the orders by ids
sample_f6['ids_cat'] = pd.Categorical(
    sample_f6['id'], 
    categories=ids, 
    ordered=True
)

sample_f6 = sample_f6.sort('ids_cat').drop('ids_cat', 1)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [164]:
# reindex
sample_f6 = sample_f6.reset_index().drop('index', 1)

In [165]:
# add sample tweet rate
sample_f6['tweet_rate'] = sample_tr

In [166]:
sample_f6.head()

,id,followers_count,following,statuses_count,ff_ratio,age,following_rate,tweet_rate
0,3301875373,47.0,716.0,844.0,15.234043,657.0,1.089802,9.93902
1,3169917217,20.0,940.0,20.0,47.000000,754.0,1.246684,None
2,2489408846,21.0,396.0,1060.0,18.857143,1102.0,0.359347,15.1884
3,2418101186,12.0,487.0,69.0,40.583333,1145.0,0.425328,2.55556
4,2500271185,25.0,672.0,2832.0,26.880000,1096.0,0.613139,18.3684


In [ ]:
# get the different ratio

In [129]:
mob_def = 'Twitter for Android' or 'Twitter for iPhone' or 'Twitter for iPad' or 'Twitter for BlackBerry\xae' or 'Twitter for Mac'
web_def = 'Twitter Web Client' or 'Twitter for Websites'
third_party = 'TweetDeck' or 'Facebook' or 'Instagram' or 'Echofon' or 'Tweetbot for i\u039fS' or 'TweetCaster for Android' or 'Hootsuite' or 'SocialOomph'

In [141]:
from collections import Counter

mobile_ratio = []
website_ratio = []
third_ratio = []
other_ratio = []

for i in range(len(sample_tw)):
    try:
        count_tw = Counter(sample_tw[i]['Source'])
        mobile_ratio.append(count_tw[mob_def] / sum(count_tw.values()))
        website_ratio.append(count_tw[web_def] / sum(count_tw.values()))
        third_ratio.append(count_tw[third_party] / sum(count_tw.values()))
        other_ratio.append(1- (count_tw[mob_def]+count_tw[web_def]+count_tw[third_party])/sum(count_tw.values()))
    except (TypeError, ZeroDivisionError):
        mobile_ratio.append('No tweets')
        website_ratio.append('No tweets')
        third_ratio.append('No tweets')
        other_ratio.append('No tweets')

In [119]:
a = Counter()
for i in range(len(sample_tw)):
    try:
        a += Counter(sample_tw[i]['Source'])
    except TypeError:
        pass

In [169]:
sample_f6['mobile_ratio'] = mobile_ratio

In [173]:
sample_f6['website_ratio'] = website_ratio
sample_f6['third_ratio'] = third_ratio
sample_f6['other_ratio'] = other_ratio

In [182]:
basic_f = pd.read_csv('./Dataset/update_data.csv', index_col = 0)
sample_basic = basic_f[basic_f.id.isin(ids)]

In [183]:
sample_basic['ids_cat'] = pd.Categorical(
    sample_basic['id'], 
    categories=ids, 
    ordered=True
)

sample_basic = sample_basic.sort('ids_cat').drop('ids_cat', 1)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [198]:
final_sample = pd.merge(sample_basic, sample_f6, on = 'id')

In [199]:
final_sample = final_sample[['id', 'user_name', 'screen_name', 'statuses_count_x',
       'followers_count_x', 'friends_count', 'favourites_count',
       'listed_count', 'url', 'lang', 'time_zone', 'location',
       'default_profile', 'default_profile_image', 'geo_enabled',
       'profile_image_url', 'profile_use_background_image',
       'profile_background_image_url_https', 'profile_text_color',
       'profile_image_url_https', 'profile_sidebar_border_color',
       'profile_background_tile', 'profile_sidebar_fill_color',
       'profile_background_image_url', 'profile_background_color',
       'profile_link_color', 'utc_offset', 'is_translator',
       'follow_request_sent', 'protected', 'verified', 'notifications',
       'description', 'contributors_enabled', 'following_x', 'created_at', 
       'followers_count_y', 'following_y', 'statuses_count_y',
       'ff_ratio', 'age', 'following_rate', 'tweet_rate', 'mobile_ratio',
       'website_ratio', 'third_ratio', 'other_ratio', 'label']]

In [232]:
url = ['url', 'profile_image_url', 'profile_background_image_url_https', 'profile_image_url_https', 'profile_background_image_url']

In [233]:
for i in url:
    final_sample[i] = np.where(final_sample[i].str.contains('http') == True, 1, 0)

In [237]:
final_sample.to_csv('sample_feature_guanyu.csv', encoding = 'utf-8')